<a href="https://colab.research.google.com/github/bogdanbabych/a05mteval/blob/main/bleuFileV04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BLEU: BiLingual Evaluation Understudy

*NLP evaluation metric used in Machine Translation tasks*

*Suitable for measuring corpus level similarity*

*$n$-gram comparison between words in candidate sentence and reference sentences*

*Range: 0 (no match) to 1 (exact match)*

### 1. Libraries
*Install and import necessary libraries*


In [1]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 2. Dataset
*Array of words: candidate and reference sentences split into words*

In [12]:
hyp1 = str('The fat cat was comfortably sitting on a big mat').split()
hyp2 = str('The fat cat was comfortably sitting on a big rug').split()
hyp3 = str('The clever cat was comfortably sitting on a small pillow').split()
hyp4 = str('The big dog was comfortably sitting on a small mat').split()


ref_a = str('The fat cat was comfortably sitting on a big rug').split()
ref_b = str('The plump cat lounged cozily on a large mat').split()
ref_c = str('A chubby feline was resting peacefully atop a spacious rug').split()
ref_d = str('The hefty cat relaxed contentedly on a broad mat').split()


### 3. *Sentence* score calculation
*Compares 1 hypothesis (candidate or source sentence) with 1+ reference sentences, returning the highest score when compared to multiple reference sentences.*

In [ ]:
score_ref_a = bleu.sentence_bleu([ref_a], hyp1)
print("Hyp1 and ref_a : {}".format(score_ref_a))



In [ ]:
score_ref_b = bleu.sentence_bleu([ref_a], hyp2)
print("Hyp and ref_b : {}".format(score_ref_b))


In [ ]:
score_ref_b = bleu.sentence_bleu([ref_a], hyp3)
print("Hyp and ref_b : {}".format(score_ref_b))

In [ ]:
score_ref_b = bleu.sentence_bleu([ref_a], hyp4)
print("Hyp and ref_b : {}".format(score_ref_b))

In [ ]:
score_ref_ab = bleu.sentence_bleu([ref_a, ref_b, ref_c, ref_d], hyp4)
print("Hyp vs multiple refs: {}".format(score_ref_ab))

In [17]:
hyp1

['The', 'fat', 'cat', 'was', 'comfortably', 'sitting', 'on', 'a', 'big', 'mat']

# Test the following sentences:


## Reference:

"More than ever, we need to stand up for science. Science that is universal – shared by all humanity – and that is unifying," the president of the European Commission said on Monday in a speech delivered at La Sorbonne University in Paris.


## Original (DE):

"Mehr denn je müssen wir für die Wissenschaft eintreten. Eine Wissenschaft, die universell ist - von der ganzen Menschheit geteilt - und die uns vereint", sagte der Präsident der Europäischen Kommission am Montag in einer Rede an der Universität La Sorbonne in Paris.

### hypothesis 1 (Google)
"More than ever, we must stand up for science. A science that is universal — shared by all humanity — and that unites us," said the President of the European Commission in a speech at La Sorbonne University in Paris on Monday.


### hypotesis 2 (deepL)

“More than ever, we must stand up for science. A science that is universal—shared by all of humanity—and that unites us,” said the President of the European Commission on Monday in a speech at La Sorbonne University in Paris.

### hypothesis 3 (chatGPT)

"More than ever, we must stand up for science. A science that is universal – shared by all of humanity – and that unites us," said the President of the European Commission on Monday in a speech at the University of La Sorbonne in Paris.

### hypothesis 4 (systran)

"More than ever, we need to stand up for science. A science that is universal - shared by all humanity - and that unites us", said the President of the European Commission in a speech at the University of La Sorbonne in Paris on Monday.

## Original (UK)


«Сьогодні, як ніколи раніше, ми повинні відстоювати науку. Науку, яка є універсальною, спільною для всього людства і яка об’єднує», — сказала голова Європейської комісії у своїй промові в університеті Сорбонна в Парижі.

### hypothesis 5 (Google (uk>en))

"Today, more than ever, we must stand up for science. Science that is universal, common to all humanity and that unites," said the European Commission President in her speech at the Sorbonne University in Paris.

